In [9]:
import spacy
import en_core_web_sm
import nltk
from nltk.corpus import wordnet as wn
import numpy as np
import pandas as pd
import polars as pl

In [2]:
nlp = en_core_web_sm.load()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adamlear/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
description = "An adorable, tuxedo-clad bird with a distinctive waddle, often found in icy habitats, known for its expert swimming skills and social nature."
description_embedding = nlp(description).vector
pos = 'n'

In [22]:
def produce_csv(pos, file_name):
    df = pd.DataFrame()
    synset_list = []
    embedding_list = []
    for synset in wn.all_synsets(pos):
        synset_list.append(synset)
        embedding_list.append(nlp(synset.definition()).vector)

    df['synset'] = synset_list
    df['embedding'] = embedding_list

    df.to_csv(file_name)

def produce_csv(pos, file_name):
    synset_list = []
    embedding_list = []

    for synset in wn.all_eng_synsets(pos):
        synset_list.append(synset)
        embedding_list.append(nlp(synset.definition()).vector)

    data = {'synset': synset_list, 'embedding': embedding_list}
    df = pl.DataFrame(data)

    df.write_csv(file_name)


In [23]:
produce_csv('n', 'nouns.csv')

ComputeError: csv writer does not support object dtype

In [ ]:
produce_csv('v', 'verbs.csv')
produce_csv('a', 'adjectives.csv')
produce_csv('r', 'adverbs.csv')

In [19]:
data = pl.read_csv('nouns.csv', ignore_errors=True)

best = None
highest = -2
for row in data.iter_rows():
    similarity = np.dot(np.array(row[2]), description_embedding) 
    if similarity > highest:
        best = row[1]
        highest = similarity

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U1159'), dtype('<U32')) -> None